# Jadwal Praktikum Generator

## Import libraries

In [ ]:
import itertools
import json
import random

## Create General Constants 

In [ ]:
DAYS = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu']
SESSIONS = [1, 2, 3, 4]
ROOMS = ['0604', '0605', '0617', '0618', '0704', '0705', '0712', '0713']

ADDITIONAL_UNAVAILABLE_GENERAL_SCHEDULES = [
    ('Jumat', 2, '0604'),
    ('Jumat', 2, '0605'),
    ('Jumat', 2, '0617'),
    ('Jumat', 2, '0618'),
    ('Jumat', 2, '0704'),
    ('Jumat', 2, '0705'),
    ('Jumat', 2, '0712'),
    ('Jumat', 2, '0713'),

    ('Jumat', 4, '0604'),
    ('Jumat', 4, '0605'),
    ('Jumat', 4, '0617'),
    ('Jumat', 4, '0618'),
    ('Jumat', 4, '0704'),
    ('Jumat', 4, '0705'),
    ('Jumat', 4, '0712'),
    ('Jumat', 4, '0713'),

    ('Sabtu', 4, '0604'),
    ('Sabtu', 4, '0605'),
    ('Sabtu', 4, '0617'),
    ('Sabtu', 4, '0618'),
    ('Sabtu', 4, '0704'),
    ('Sabtu', 4, '0705'),
    ('Sabtu', 4, '0712'),
    ('Sabtu', 4, '0713'),
]

ADDITIONAL_AVAILABLE_GENERAL_SCHEDULES = [ ]

AVAILABLE_GENERAL_SCHEDULES = [
    schedule for schedule in itertools.product(DAYS, SESSIONS, ROOMS)
    if schedule not in ADDITIONAL_UNAVAILABLE_GENERAL_SCHEDULES
] + ADDITIONAL_AVAILABLE_GENERAL_SCHEDULES

## Load Input Data

In [ ]:
with open('input.json', 'r') as file:
    classes = json.load(file)

## Populate Avaliable Schedules for All Classes

In [ ]:
for i, cls in enumerate(classes):
    class_unavailable_schedules = [(s['day'], s['session'], s['room'])
                                  for s in cls.get('unavailable_schedules')]

    class_available_schedules = [s for s in AVAILABLE_GENERAL_SCHEDULES
                                 if s not in class_unavailable_schedules]

    classes[i].setdefault('available_schedules', class_available_schedules)

    del classes[i]['unavailable_schedules']

## Assign Schedules using DFS

In [ ]:
shuffled_classes = random.sample(classes, len(classes))
lecturer_schedule = set()
schedules_out = {str(schedule): None for schedule in AVAILABLE_GENERAL_SCHEDULES}


def backtrack(class_idx):
    if class_idx == len(shuffled_classes):
        return True

    current_class = shuffled_classes[class_idx]

    for schedule in current_class['available_schedules']:
        day, session, _ = schedule
        schedule_key = str(schedule)
        lecturer_schedules = (current_class['lecturer_code'], day, session)

        if schedules_out.get(schedule_key) is None and lecturer_schedules not in lecturer_schedule:
            schedules_out[schedule_key] = {
                'name': current_class['name'],
                'subject': current_class['subject'],
                'lecturer_code': current_class['lecturer_code'],
            }
            lecturer_schedule.add(lecturer_schedules)

            if backtrack(class_idx + 1):
                return True

            schedules_out[schedule_key] = None
            lecturer_schedule.remove(lecturer_schedules)

    return False


if backtrack(0):
    with open("output.json", "w") as file:
        json.dump(schedules_out, file, indent=4)